# **Refit Pipeline, Model and Evaluate**

## Objectives

* Here, we will refit the pipeline with the most important features
* Fit and evaluate the ML pipeline to predict attrition

## Inputs

* Raw dataset in outputs/datasets/collection/employee-attrition.csv

## Outputs

* TrainSet and TestSet
* Data cleaning and feature engineering pipeline
* Modeling pipeline

---

# Change working directory

In [ ]:
import os
current_dir = os.getcwd()
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
current_dir

---

# Load the dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
df = (pd.read_csv(f"outputs/datasets/collection/employee-attrition.csv")
        .drop(labels=['DailyRate','EmployeeCount', 'EmployeeNumber', 'HourlyRate',
                      'MonthlyRate', 'StandardHours', 'Over18'], axis=1))

df['Attrition'] = df['Attrition'].replace({"Yes":1, "No":0})

df.head()

From the last notebook, these are the most important features to predict attrition. We will filter the dataset such that it contains only those features.


In [ ]:
best_features_and_target = [
                            'OverTime', 'YearsAtCompany', 
                            'Age', 'JobLevel', 'EnvironmentSatisfaction',
                            'Department', 'JobSatisfaction', 'StockOptionLevel',
                            'JobInvolvement', 'Attrition'
                            ]

df = df.filter(best_features_and_target)
df.head()


Next, we will create:
* Split the dataset
* Data cleaning and feature engineering pipeline
* Handle imbalance
* Modeling pipeline

---

# Split the dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Attrition'], axis=1),
                                                    df['Attrition'],
                                                    test_size=0.2,
                                                    random_state=0,
                                                   )

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


---

## ML Pipeline 1: Data cleaning and feature engineering pipeline

We create the datacleaning and feature engineering pipeline based on the conclusions from the last notebook.

In [ ]:
from sklearn.pipeline import Pipeline

# Feature Engineering
from feature_engine.selection import SmartCorrelatedSelection
from feature_engine.encoding import OrdinalEncoder
from feature_engine import transformation as vt


def PipelineDataCleaningAndFeatureEngineering():
    pipeline_base = Pipeline([
        ('yj', vt.YeoJohnsonTransformer(variables=['YearsAtCompany']) ),
        ('OrdinalCategoricalEncoder', OrdinalEncoder(encoding_method='arbitrary',
                                                     variables=['Department', 'OverTime'])),
    ])

    return pipeline_base


PipelineDataCleaningAndFeatureEngineering()


### Fit the pipeline

In [ ]:
pipeline_data_cleaning_feat_eng = PipelineDataCleaningAndFeatureEngineering()
X_train = pipeline_data_cleaning_feat_eng.fit_transform(X_train)
X_test = pipeline_data_cleaning_feat_eng.transform(X_test)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

---

# Handle imbalance

Recap on the target imbalance mentioned in notebook 02. There is more 'No attrition' or 0's than 'Yes attrition' or 1's.

In [ ]:
attrition_array = y_train.to_numpy()

def plot_target_count(target):
    fig, axes = plt.subplots(figsize=(5,5))
    sns.countplot(x = target)
    plt.show()

plot_target_count(attrition_array)

We will use Synthetic Minority Over-sampling Technique (SMOTE) from the `imblearn` library. The sampling strategy will be `minority` so it resamples all classes except the majority.

In [ ]:
from imblearn.over_sampling import SMOTE

smote_over = SMOTE(sampling_strategy='minority', random_state=0)
X_train, y_train = smote_over.fit_resample(X_train, y_train)

We plot once again to check after SMOTE operation.

In [ ]:
attrition_array_balanced = y_train.to_numpy()
plot_target_count(attrition_array_balanced)

And we notice that the traget is now balanced. One has to mind that we changed the dataset for the benefit of fitting a better model.

---

## ML Pipeline2: Modelling and Hyperparameter Optimisation pipeline

Here, the pipeline consists of:
* **StandardScaler**: to rescale the features to have standard normal distribution with zero mean and standard deviation of 1. It is performed on all variables. The variable distribution might be slightly different.
* **SelectFromModel**: to select the relevant features for fitting. We use the embedded method to perform feature selection during training. The model will be the algorithm of our choice.
* **model**: the ML algorithm

In [ ]:
# Feat Scaling
from sklearn.preprocessing import StandardScaler
# Feat Selection
from sklearn.feature_selection import SelectFromModel
# ML algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier


def PipelineClf(model):
    pipeline_base = Pipeline([
        ("scaler", StandardScaler()),
        ("model", model),
    ])

    return pipeline_base

### Grid search Cross Validation

We use the custom python class for hyperparameter optimization provided by CI.

In [ ]:
from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:
    """
    Custom class provided by CI
    """
    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")

            model = PipelineClf(self.models[key])
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, )
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)
        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        return df[columns], self.grid_searches

We use the best classifier model (GradientBoostingClassifier) with the best hyperparameters

In [ ]:
best_model = {
    "GradientBoostingClassifier": GradientBoostingClassifier(random_state=0),
}

best_params = {
    "GradientBoostingClassifier": {
        'model__max_depth': [3],
        'model__min_samples_split': [2],
        'model__min_weight_fraction_leaf': [0.0],
        'model__n_estimators': [1000],
    },
}

Fit the pipeline to the TrainSet data. We set cross validation `cv` to 5 and we use all processors or threads by setting `n_jobs` to -1.

In [ ]:
from sklearn.metrics import make_scorer, recall_score
search = HyperparameterOptimizationSearch(models=best_model, params=best_params)
search.fit(X_train, y_train,
           scoring =  make_scorer(recall_score, pos_label=1),
           n_jobs=-1, cv=5)

Get the performance summary

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary 

Grab the pipeline

In [ ]:
best_model = grid_search_summary.iloc[0,0]
pipeline_clf = grid_search_pipelines[best_model].best_estimator_
pipeline_clf

### Assess feature importnace

In [ ]:
# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': X_train.columns,
    'Importance': pipeline_clf['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# re-assign best_features order
best_features = df_feature_importance['Feature'].to_list()

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{best_features}")

sns.barplot(data=df_feature_importance, x="Feature", y="Importance")
plt.xticks(rotation=90)

From the 27 features, we end up with 9 important features with `OverTime` being the variable that strongly affects the target. We will use all 9 features for refitting the model. But first we evaluate the current pipeline.

---

## Evaluate the pipeline

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix


def confusion_matrix_and_report(X, y, pipeline, label_map):

    prediction = pipeline.predict(X)

    print('---  Confusion Matrix  ---')
    print(pd.DataFrame(confusion_matrix(y_true=prediction, y_pred=y),
          columns=[["Actual " + sub for sub in label_map]],
          index=[["Prediction " + sub for sub in label_map]]
          ))
    print("\n")

    print('---  Classification Report  ---')
    print(classification_report(y, prediction, target_names=label_map), "\n")


def clf_performance(X_train, y_train, X_test, y_test, pipeline, label_map):
    print("#### Train Set #### \n")
    confusion_matrix_and_report(X_train, y_train, pipeline, label_map)

    print("#### Test Set ####\n")
    confusion_matrix_and_report(X_test, y_test, pipeline, label_map)

In [ ]:
clf_performance(X_train=X_train, y_train=y_train,
                X_test=X_test, y_test=y_test,
                pipeline=pipeline_clf,
                label_map= ['No Attrition', 'Attrition'] 
                )

We see that most of the model success metrics are met. These were:
* A minimum of 80% no-attrition on train and test datasets, here we get 99% and 88%, respectively
* A minimum of 60% attrition on train and test datasets, here we get 99% and 40%, respectively
* Note that the attrition prediction still does not meet the requirements (20% below). Ideally, further iterations with other models and hyperparameters can be tested. However, due to time limitation of the project, we will not proceed with further iterations.

---

## Dump the model pipeline

In [ ]:
import joblib

version = 'v3'
file_path = f'outputs/ml_pipelines/{version}'

try:
    os.makedirs(name=file_path)
except Exception as e:
    print(e)

Save the train and test datasets

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)
y_train.to_csv(f"{file_path}/y_train.csv", index=False)
X_test.to_csv(f"{file_path}/X_test.csv", index=False)
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

Dump the pipelines to pickle files

In [ ]:
# Save the data cleaning and feature engineering pipeline
joblib.dump(value=pipeline_data_cleaning_feat_eng ,
            filename=f"{file_path}/clf_pipeline_data_cleaning_feat_eng.pkl")
# Save the modeling pipeline
joblib.dump(value=pipeline_clf ,
            filename=f"{file_path}/clf_pipeline_model.pkl")

Feature importance was one of the business requirement, therefore, we plot it and save the file.

In [ ]:
sns.barplot(data=df_feature_importance, x="Feature", y="Importance")
plt.xticks(rotation=90)
plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')

We notice that 5 of the 9 variables remained in the same ranking from the last iteration (previous notebook). Namely, the first 4 variables and `Department`. The other 4 variables have shuffled in between each other.

---

# Conclusions

* The final data cleaning and feature engineering as well as modeling pipelines were generated
* The model can predict no-attrition reliably but other iteratons or more data in the dataset is needed to predict attrition better